회귀분석 시 변수들 간의 다중공선성 문제가 있을 때 주성분을 통한 회귀분석 필요

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression, PLSSVD
from sklearn.metrics import mean_squared_error

## 다중공선성확인

In [340]:
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
data=pd.read_excel('D:/2018-2_ck project/final.xlsx')

In [ ]:
data.columns

In [ ]:
var="broad_kbs+broad_mbc+broad_sbs+broad_jtbc+broad_tvn+broad_mnet+broad_etc+age_12+age_15+age_19+age_all+week_mon+week_tue+week_wed+week_thu+week_fri+week_sat+week_sun+tw_gold+tw_week+tw_etc+ca_mc+ca_gag+ca_tal+ca_sing+ca_act+ca_bro+ca_etc+log_award+log_max+log_pro1+log_pro2+log_pd_award+log_pd_max+log_pd_pro1+log_pd_pro2+portal_naver+portal_daum+pd_portal"

In [366]:
y,X=dmatrices("logy~"+var,data=data, return_type="dataframe")

In [ ]:
vif=[variance_inflation_factor(X.values,i) for i in range(x.shape[1])]

In [ ]:
result=sm.OLS(y,X).fit()
print(result.summary())

In [370]:
#p-value 0.5넘을 경우 다중공선성 의심
# broad_mbc, broad_mnet, age_12, age_19, week_mon, week_tue, week_wed, ca_gag, ca_bro, log_pro1, log_pd_award, log_pd_pro1, log_pd_pro2  의심

# PCR

In [432]:
train = pd.read_excel('D:/2018-2_ck project/final_train.xlsx')
test= pd.read_excel('D:/2018-2_ck project/final_test.xlsx')

In [433]:
x_train=train.iloc[:,1:-2]
y_train=train.iloc[:,-1]

x_test=test.iloc[:,1:-2]
y_test=test.iloc[:,-1]

In [ ]:
pca2 = PCA()
X_reduced_train = pca2.fit_transform(scale(x_train))

각각의 연속적인 주성분을 더함으로써 설명되는 분산의 양을 얻기 위해 약간의 계산을 함

In [ ]:
np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

MSE에 어떻게 영향을 주는지 확인하기 위해 10 배 교차 유효성 검사를 수행

In [438]:
# 10 배 CV, 셔플 사용
n = len(X_reduced_train)
kf_10 = model_selection.KFold( n_splits=10, shuffle=True, random_state=1)

regr = LinearRegression()
mse = []

In [ ]:
# 절편만있는 MSE 계산 (회귀 분석에서 주요 구성 요소 없음)
score = -1*model_selection.cross_val_score(regr, np.ones((n,1)), y_train.ravel(), cv=kf_10, scoring='neg_mean_squared_error').mean()    
mse.append(score)
score

In [440]:
# 19 가지 주요 구성 요소에 대해 CV를 사용하여 MSE를 계산하고 그 때 한 구성 요소를 추가
for i in np.arange(1, 25):
    score = -1*model_selection.cross_val_score(regr, X_reduced_train[:,:i], y_train.ravel(), cv=kf_10, scoring='neg_mean_squared_error').mean()
    mse.append(score)

In [ ]:
plt.plot(mse, '-v')
plt.xlabel('Number of principal components in regression')#'회귀 분석의 주요 구성 요소 수' 
plt.ylabel('MSE')
plt.title('y')
plt.xlim(xmin=-1);

가장 낮은 유효성 검사 오류는  M=19,20 구성 요소가 사용될 때 발생합니다. 이제 테스트 데이터에서 어떻게 수행되는지 확인하고 다음과 같이 테스트 MSE를 계산합니다.

In [442]:
#M=19
X_reduced_test = pca2 .  transform ( scale( x_test )) [:, : 20]

# 훈련 데이터에 대한 회귀 모델 모델링
regr = LinearRegression()
regr.fit(X_reduced_train[:,:20], y_train)  

# 테스트 데이터로 예측 및 mse, rmse, r^2 
pred = regr.predict(X_reduced_test)
print("Mean Squared Error:", metrics.mean_squared_error(y_test,y_predict))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test,y_predict)))
sst=sum((y_test-pred+pred-np.mean(y_test))**2)
ssr=sum((pred-np.mean(y_test))**2)
r_square=(ssr/sst)
print("R^2:",r_square)

In [447]:
#M=20
X_reduced_test = pca2 .  transform ( scale( x_test )) [:, : 21]
    
# 훈련 데이터에 대한 회귀 모델 모델링
regr = LinearRegression()
regr.fit(X_reduced_train[:,:21], y_train)

# 테스트 데이터로 예측 및 mse, rmse, r^2 
pred = regr.predict(X_reduced_test)
print("Mean Squared Error:", metrics.mean_squared_error(y_test,y_predict))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test,y_predict)))
sst=sum((y_test-pred+pred-np.mean(y_test))**2)
ssr=sum((pred-np.mean(y_test))**2)
r_square=(ssr/sst)
print("R^2:",r_square)